## 📚 01 · Introduction to Ray Train  
In this notebook you’ll learn how to run **distributed data-parallel training with PyTorch** on an Anyscale cluster using **Ray Train V2**. You’ll train a **ResNet-18 model on MNIST** across multiple GPUs, with built-in support for **checkpointing, metrics reporting, and distributed orchestration**.  

### What you’ll learn & take away
* Why and when to use **Ray Train** for distributed training instead of managing PyTorch DDP manually  
* How to wrap your PyTorch code with **`prepare_model()`** and **`prepare_data_loader()`** for multi-GPU execution  
* How to configure scale with **`ScalingConfig(num_workers=..., use_gpu=True)`** and track outputs with **`RunConfig(storage_path=...)`**  
* How to **report metrics and save checkpoints** using `ray.train.report(...)`, with best practices for rank-0 checkpointing  
* How to use **Anyscale storage**: fast local NVMe vs. persistent cluster/cloud storage  
* How to **inspect training results** (metrics DataFrame, checkpoints) and load a checkpointed model for inference with Ray  

> The entire workflow runs **fully distributed from the start**: you define your training loop once, and Ray handles orchestration, sharding, and checkpointing across the cluster.


###  🔎 When to use Ray Train  

Use Ray Train when you face one of the following challenges:

|Challenge|Detail|Solution|
|---|---|---|
|Need to speed up or scale up training| Training jobs might take a long time to complete, or require a lot of compute | Ray Train provides a distributed training framework that allows engineers to scale training jobs to multiple GPUs |
|Minimize overhead of setting up distributed training| Engineers need to manage the underlying infrastructure | Ray Train handles the underlying infrastructure via Ray's autoscaling |
|Achieve observability| Engineers need to connect to different nodes and GPUs to find the root cause of failures, fetch logs, traces, etc | Ray Train provides observability via Ray's dashboard, metrics, and traces that allow engineers to monitor the training job |
|Ensure reliable training| Training jobs can fail due to hardware failures, network issues, or other unexpected events | Ray Train provides fault tolerance via checkpointing, automatic retries, and the ability to resume training from the last checkpoint |
|Avoid significant code rewrite| Engineers might need to fully rewrite their training loop to support distributed training | Ray Train provides a suite of integrations with the PyTorch ecosystem, Tree-based methods (XGB, LGBM), and more to minimize the amount of code changes needed |

### 🖥️ How Distributed Data Parallel (DDP) Works  

The diagram above shows the **lifecycle of a single training step** in PyTorch DistributedDataParallel (DDP) when orchestrated by Ray Train:

1. **Model Replication**  
   The model is initialized on GPU rank 0 and broadcast to all other workers so that each has an identical copy.  

2. **Sharded Data Loading**  
   The dataset is automatically split into **non-overlapping shards**. Each worker processes only its shard, ensuring efficient parallelism without duplicate samples.  

3. **Forward & Backward Passes**  
   Each worker runs a forward pass and computes gradients locally during the backward pass.  

4. **Gradient Synchronization**  
   Gradients are aggregated across workers via an **AllReduce** step, ensuring that model updates stay consistent across all GPUs.  

5. **Weight Updates**  
   Once gradients are synchronized, each worker applies the update, keeping model replicas in sync.  

6. **Checkpointing & Metrics**  
   By convention, only the **rank 0 worker** saves checkpoints and logs metrics to persistent storage. This avoids duplication while preserving progress and results.  

With Ray Train, you don’t need to manage process groups or samplers manually—utilities like `prepare_model()` and `prepare_data_loader()` wrap these details so your code works out of the box in a distributed setting.

|<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-ai-libraries/diagrams/multi_gpu_pytorch_v4.png" width="70%" loading="lazy">|
|:--|
|Schematic overview of DistributedDataParallel (DDP) training: (1) the model is replicated from the <code>GPU rank 0</code> to all other workers; (2) each worker receives a shard of the dataset and processes a mini-batch; (3) during the backward pass, gradients are averaged across GPUs; (4) checkpoint and metrics from rank 0 GPU are saved to the persistent storage.|

## 01 · Imports  

Start by importing all the libraries you’ll need for this tutorial.  

- **Standard utilities**: `os`, `datetime`, `tempfile`, `csv`, `shutil`, and `gc` help with file paths, checkpointing, cleanup, and general housekeeping.  
- **Data and visualization**: `pandas`, `numpy`, `matplotlib`, and `PIL` are used for inspecting the dataset and plotting sample images.  
- **PyTorch**: core deep learning components (`torch`, `CrossEntropyLoss`, `Adam`) plus `torchvision` for loading MNIST and building a ResNet-18 model.  
- **Ray Train**: the key imports for distributed training—`ScalingConfig`, `RunConfig`, and `TorchTrainer`. These handle cluster scaling, experiment output storage, and execution of your training loop across GPUs.  

This notebook assumes Ray is already running (for example, inside an Anyscale cluster), so you don’t need to call `ray.init()` manually.

In [ ]:
# 01. Imports

# --- Standard library: file IO, paths, timestamps, temp dirs, cleanup ---
import csv            # Simple CSV logging for metrics in single-GPU section
import datetime       # Timestamps for run directories / filenames
import os             # Filesystem utilities (paths, env vars)
import tempfile       # Ephemeral dirs for checkpoint staging with ray.train.report()
import shutil         # Cleanup of artifacts (later cells)
import gc             # Manual garbage collection to cleanup after inference

from pathlib import Path  # Convenient, cross-platform path handling

# --- Visualization & data wrangling ---
import matplotlib.pyplot as plt  # Plot sample digits and metrics curves
from PIL import Image            # Image utilities for inspection/debug
import numpy as np               # Numeric helpers (random sampling, arrays)
import pandas as pd              # Read metrics.csv into a DataFrame

# --- PyTorch & TorchVision (model + dataset) ---
import torch
from torch.nn import CrossEntropyLoss      # Classification loss for MNIST
from torch.optim import Adam               # Optimizer
from torchvision.models import resnet18    # Baseline CNN (we’ll adapt for 1-channel input)
from torchvision.datasets import MNIST     # Dataset
from torchvision.transforms import ToTensor, Normalize, Compose  # Preprocessing pipeline

# --- Ray Train (distributed orchestration) ---
import ray
from ray.train import ScalingConfig, RunConfig      # Configure scale and storage
from ray.train.torch import TorchTrainer            # Multi-GPU PyTorch trainer (DDP/FSDP)


### 02 · Download MNIST Dataset  

Next, download the **MNIST dataset** using `torchvision.datasets.MNIST`.  
- This will automatically fetch the dataset (if not already present) into a local `./data` directory.  
- MNIST consists of **60,000 grayscale images of handwritten digits (0–9)**, each sized **28×28 pixels**.  
- By setting `train=True`, we load the training split of the dataset.  

Once downloaded, we’ll later wrap this dataset in a `DataLoader` and apply normalization so it can be used for model training.

In [ ]:
# 02. Download MNIST Dataset  

dataset = MNIST(root="/mnt/cluster_storage/data", train=True, download=True)

<div class="alert alert-info">

<b>Note about Anyscale storage options</b>

In this example, the MNIST dataset is stored under <code>/mnt/cluster_storage/</code>, which is Anyscale’s **persistent cluster storage**.  

* Unlike node-local NVMe volumes, cluster storage is **shared across nodes** in your cluster.  
* Data written here will **persist across cluster restarts**, making it a safe place for datasets, checkpoints, and results.  
* This is the recommended location for training data and artifacts you want to reuse.  

* Anyscale also provides each node with its own volume and disk and doesn’t share them with other nodes.
* Local storage is very fast - Anyscale supports the Non-Volatile Memory Express (NVMe) interface.
* Local storage is not a persisent storage, Anyscale deletes data in the local storage after instances are terminated. 

Read more about available <a href="https://docs.anyscale.com/configuration/storage" target="_blank">storage</a> options.
</div>

### 03 · Visualize Sample Digits  

Before training, let’s take a quick look at the dataset.  
- We’ll randomly sample **9 images** from the MNIST training set.  
- Each image is a **28×28 grayscale digit**, with its ground-truth label shown above the plot.  
- This simple visualization is a good sanity check to confirm that the dataset downloaded correctly and that labels align with the images.  

In [ ]:
# 03. Visualize Sample Digits

# Create a square figure for plotting 9 samples (3x3 grid)
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3

# Loop through grid slots and plot a random digit each time
for i in range(1, cols * rows + 1):
    # Randomly select an index from the dataset
    sample_idx = np.random.randint(0, len(dataset.data))
    img, label = dataset[sample_idx]  # image (PIL) and its digit label
    
    # Add subplot to the figure
    figure.add_subplot(rows, cols, i)
    plt.title(label)         # show the digit label above each subplot
    plt.axis("off")          # remove axes for cleaner visualization
    plt.imshow(img, cmap="gray")  # display as grayscale image

## 04 · Define ResNet-18 Model for MNIST  

Now let’s define the **ResNet-18** architecture we’ll use for classification.  

- `torchvision.models.resnet18` is preconfigured for **3-channel RGB input** and **ImageNet classes**.  
- Since MNIST digits are **1-channel grayscale** images with **10 output classes**, we need two adjustments:  
  1. Override the first convolution layer (`conv1`) to accept **`in_channels=1`**.  
  2. Set the final layer to output **10 logits**, one per digit class (handled by `num_classes=10`).  

This gives us a ResNet-18 tailored for MNIST while preserving the rest of the architecture.

In [ ]:
# 04. Define ResNet-18 Model for MNIST

def build_resnet18():
    # Start with a torchvision ResNet-18 backbone
    # Set num_classes=10 since MNIST has digits 0–9
    model = resnet18(num_classes=10)

    # Override the first convolution layer:
    # - Default expects 3 channels (RGB images)
    # - MNIST is grayscale → only 1 channel
    # - Keep kernel size/stride/padding consistent with original ResNet
    model.conv1 = torch.nn.Conv2d(
        in_channels=1,   # input = grayscale
        out_channels=64, # number of filters remains the same as original ResNet
        kernel_size=(7, 7),
        stride=(2, 2),
        padding=(3, 3),
        bias=False,
    )

    # Return the customized ResNet-18
    return model

<div class="alert alert-block alert-info">

<b>Migration roadmap: from standalone PyTorch to PyTorch with Ray Train</b>  

The following are the steps to take a **regular PyTorch training loop** and run it in a **fully distributed setup with Ray Train**.  

<ol>
    <li><b>Configure scale and GPUs</b> — decide how many workers and whether each should use a GPU.</li>
    <li><b>Wrap the model with Ray Train</b> — use <code>prepare_model()</code> to move the ResNet to the right device and wrap it in DDP automatically.</li>
    <li><b>Wrap the dataset with Ray Train</b> — use <code>prepare_data_loader()</code> so each worker gets a distinct shard of MNIST, moved to the correct device.</li>
    <li><b>Add metrics & checkpointing</b> — report training loss and save checkpoints with <code>ray.train.report()</code> from rank-0.</li>
    <li><b>Configure persistent storage</b> — store outputs under <code>/mnt/cluster_storage/</code> so that results and checkpoints are available across the cluster.</li>
</ol>

</div>


Ray Train is built around [four key concepts](https://docs.ray.io/en/latest/train/overview.html):
1. **Training function**: (implemented above `train_loop_ray_train`): A Python function that contains your model training logic.
1. **Worker**: A process that runs the training function.
1. **Scaling config**: specifices number of workers and compute resources (CPUs or GPUs, TPUs).
1. **Trainer**: A Python class (Ray Actor) that ties together the training function, workers, and scaling configuration to execute a distributed training job.

|<img src="https://docs.ray.io/en/latest/_images/overview.png" width="60%" loading="lazy">|
|:--|
|High-level architecture of how Ray Train|

## 05 · Define the Ray Train Loop (DDP per-worker)

This is the **per-worker training function** that Ray executes on each process/GPU. It keeps your PyTorch code intact while Ray handles **process launch, device placement, and data sharding**.

Key points:
- **Inputs via `config`**: we pass hyperparameters like `num_epochs` and a **`global_batch_size`**.
- **Model & optimizer**: `load_model_ray_train()` returns a model already wrapped by Ray Train (DDP + correct device). We use `Adam` and `CrossEntropyLoss` for MNIST.
- **Batch sizing**: we split the global batch across workers:  
  `per_worker_batch = global_batch_size // world_size`.
- **Data sharding**: `build_data_loader_ray_train(...)` returns a DataLoader wrapped with a **DistributedSampler**; each worker sees a disjoint shard.
- **Epoch control**: `data_loader.sampler.set_epoch(epoch)` ensures proper shuffling across epochs in distributed mode.
- **Training step**: standard PyTorch loop—forward → loss → zero_grad → backward → step.
- **Metrics & checkpointing**: `print_metrics_ray_train(...)` logs loss; `save_checkpoint_and_metrics_ray_train(...)` calls `ray.train.report(...)` (rank-0 saves the checkpoint).

This function is passed to `TorchTrainer`, which runs it **concurrently on all workers**.

Let's see how this data-parallel training loop will look like with Ray Train and PyTorch.

In [ ]:
# 05. Define the Ray Train per-worker training loop

def train_loop_ray_train(config: dict):  # pass in hyperparameters in config
    # config holds hyperparameters passed from TorchTrainer (e.g. num_epochs, global_batch_size)

    # Define loss function for MNIST classification
    criterion = CrossEntropyLoss()

    # Build and prepare the model for distributed training.
    # load_model_ray_train() calls ray.train.torch.prepare_model()
    # → moves model to GPU and wraps it in DistributedDataParallel (DDP).
    model = load_model_ray_train()

    # Standard optimizer (learning rate fixed for demo)
    optimizer = Adam(model.parameters(), lr=1e-5)

    # Calculate the batch size for each worker
    global_batch_size = config["global_batch_size"]
    world_size = ray.train.get_context().get_world_size()  # total # of workers in the job
    batch_size = global_batch_size // world_size  # split global batch evenly
    print(f"{world_size=}\n{batch_size=}")

    # Wrap DataLoader with prepare_data_loader()
    # → applies DistributedSampler (shards data across workers)
    # → ensures batches are automatically moved to correct device
    data_loader = build_data_loader_ray_train(batch_size=batch_size)

    # ----------------------- Training loop ----------------------- #
    for epoch in range(config["num_epochs"]):

        # Ensure each worker shuffles its shard differently every epoch
        data_loader.sampler.set_epoch(epoch)

        # Iterate over batches (sharded across workers)
        for images, labels in data_loader:
            outputs = model(images)            # forward pass
            loss = criterion(outputs, labels)  # compute loss
            optimizer.zero_grad()              # reset gradients

            loss.backward()   # backward pass (grads averaged across workers via DDP)
            optimizer.step()  # update model weights

        # After each epoch: report loss and log metrics
        metrics = print_metrics_ray_train(loss, epoch)

        # Save checkpoint (only rank-0 worker persists the model)
        save_checkpoint_and_metrics_ray_train(model, metrics)

<div class="alert alert-block alert-info">

<b>Main training loop</b>
<ul>
  <li><strong>global_batch_size</strong>: the total number of samples processed in a single training step of the entire training job.
    <ul>
      <li>It's estimated like this: <code>batch size * DDP workers * gradient accumulation steps</code>.</li>
    </ul>
  </li>
  <li>Notice that images and labels are no longer manually moved to device (<code>images.to("cuda")</code>). This is done by 
    <a href="https://docs.ray.io/en/latest/train/api/doc/ray.train.torch.prepare_data_loader.html#ray-train-torch-prepare-data-loader" target="_blank">
      prepare_data_loader()
    </a>.
  </li>
  <li>Config that will be passed here, is defined below. It will be passed to the Ray Train's <a href="https://docs.ray.io/en/latest/train/api/doc/ray.train.torch.TorchTrainer.html#ray-train-torch-torchtrainer" target="_blank">TorchTrainer</a>.</li>
  <li>
    <a href="https://docs.ray.io/en/latest/train/api/doc/ray.train.v2.api.context.TrainContext.html#ray-train-v2-api-context-traincontext" target="_blank">
      TrainContext
    </a> lets users get useful information about the training i.e. node rank, world size, world rank, experiment name.
  </li>

  <li><code>load_model_ray_train</code> and <code>build_data_loader_ray_train</code> are implemented below.</li>
</ul>
</div>

## 06 · Define `train_loop_config`  

The `train_loop_config` is a simple **dictionary of hyperparameters** that Ray passes into your training loop (`train_loop_ray_train`).  

- It acts as the **bridge between the `TorchTrainer` and your per-worker training code**.  
- Anything defined here becomes available inside the `config` argument of `train_loop_ray_train`.  

In this example we define:  
- **`num_epochs`** → how many full passes through the dataset to run.  
- **`global_batch_size`** → the total batch size across all workers (Ray will split this evenly across GPUs).  

You can add other parameters here (like `learning_rate`, `embedding_dim`, etc.) and they’ll automatically be accessible in your training loop via `config["param_name"]`.  


In [ ]:
# 06. Define the configuration dictionary passed into the training loop

# train_loop_config is provided to TorchTrainer and injected into
# train_loop_ray_train(config) as the "config" argument.
# → Any values defined here are accessible inside the training loop.

train_loop_config = {
    "num_epochs": 2,           # Number of full passes through the dataset
    "global_batch_size": 128   # Effective batch size across ALL workers
                               # (Ray will split this evenly per worker, e.g.
                               # with 8 workers → 16 samples/worker/step)
}


## 07 · Configure Scaling with `ScalingConfig`  

The `ScalingConfig` tells Ray Train **how many workers to launch** and **what resources each worker should use**.  

- **`num_workers=8`** → Run the training loop on 8 parallel workers. Each worker runs the same code on a different shard of the data.  
- **`use_gpu=True`** → Assign one GPU per worker. If you set this to `False`, each worker would train on CPU instead.  

This declarative config is what allows Ray to handle cluster orchestration for you — you don’t need to manually start processes or set CUDA devices.  

Later, we’ll pass this `scaling_config` into the `TorchTrainer` to launch distributed training.  

In [ ]:
# 07. Configure the scaling of the training job

# ScalingConfig defines how many parallel training workers Ray should launch
# and whether each worker should be assigned a GPU or CPU.
# → Each worker runs train_loop_ray_train(config) independently,
#    with Ray handling synchronization via DDP under the hood.

scaling_config = ScalingConfig(
    num_workers=8,   # Launch 8 training workers (1 process per worker)
    use_gpu=True     # Allocate 1 GPU to each worker
)

<div class="alert alert-block alert-info">

Docs on <a href="https://docs.ray.io/en/latest/train/api/doc/ray.train.ScalingConfig.html#ray-train-scalingconfig" target="_blank">ScalingConfig</a> can be found with the link in this sentence.

See docs on configuring <a href="https://docs.ray.io/en/latest/train/user-guides/using-gpus.html" target="_blank">scale and GPUs</a> for more details.
</div>

## 08 · Wrap the Model with `prepare_model()`  

Next, we define a helper function to build and prepare the model for Ray Train.  

- Start by constructing the **ResNet-18** model adapted for MNIST using `build_resnet18()`.  
- Instead of manually calling `model.to("cuda")` and wrapping it in **DistributedDataParallel (DDP)**, we use **`ray.train.torch.prepare_model()`**.  
  * This automatically:  
    - Moves the model to the correct device (GPU or CPU).  
    - Wraps it in DDP or FSDP.  
    - Ensures gradients are synchronized across workers.  

This means the same code works whether you’re training on **1 GPU or 100 GPUs** — no manual device placement or DDP boilerplate required.  


In [ ]:
# 08. Build and prepare the model for Ray Train

def load_model_ray_train() -> torch.nn.Module:
    model = build_resnet18()
    # prepare_model() → move to correct device + wrap in DDP automatically
    model = ray.train.torch.prepare_model(model)
    return model


<div class="alert alert-block alert-info">
  <a href="https://docs.ray.io/en/latest/train/api/doc/ray.train.torch.prepare_model.html#ray-train-torch-prepare-model" target="_blank">
    prepare_model()
  </a> allows users to specify additional parameters:
  <ul>
    <li><code>parallel_strategy</code>: "ddp", "fsdp" – wrap models in <code>DistributedDataParallel</code> or <code>FullyShardedDataParallel</code></li>
    <li><code>parallel_strategy_kwargs</code>: pass additional arguments to "ddp" or "fsdp"</li>
  </ul>
  
</div>

## 09 · Build the DataLoader with `prepare_data_loader()`  

Now let’s define a helper that builds the **MNIST DataLoader** and makes it Ray Train–ready.  

- Apply standard preprocessing:  
  * `ToTensor()` → convert PIL images to PyTorch tensors  
  * `Normalize((0.5,), (0.5,))` → center and scale pixel values  

- Construct a PyTorch `DataLoader` with batching and shuffling.  

- Finally, wrap it with [`prepare_data_loader()`](https://docs.ray.io/en/latest/train/api/doc/ray.train.torch.prepare_data_loader.html#ray-train-torch-prepare-data-loader), which automatically:  
  * Moves each batch to the correct device (GPU or CPU).  
  * Copies data from host memory to device memory as needed.  
  * Injects a PyTorch [`DistributedSampler`](https://pytorch.org/docs/stable/data.html#torch.utils.data.distributed.DistributedSampler) when running with multiple workers, so that each worker processes a unique shard of the dataset.  

This utility lets you use the **same DataLoader code** whether you’re training on one GPU or many — Ray handles the distributed sharding and device placement for you.  

In [ ]:
# 09. Build a Ray Train–ready DataLoader for MNIST

def build_data_loader_ray_train(batch_size: int) -> torch.utils.data.DataLoader:
    # Define preprocessing: convert to tensor + normalize pixel values
    transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
    # Load the MNIST training set from persistent cluster storage
    train_data = MNIST(
        root="/mnt/cluster_storage/data",
        train=True,
        download=True,
        transform=transform,
    )

    # Standard PyTorch DataLoader (batching, shuffling, drop last incomplete batch)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)

    # prepare_data_loader():
    # - Adds a DistributedSampler when using multiple workers
    # - Moves batches to the correct device automatically
    train_loader = ray.train.torch.prepare_data_loader(train_loader)
    
    return train_loader

<div class="alert alert-block alert-warning">

<b>Ray Data integration</b>

This step isn't necessary if you are integrating your Ray Train workload with Ray Data. It's especially useful if preprocessing is CPU-heavly and user wants to run preprocessing and training of separate instances.
</div>

## 10 · Report Training Metrics  

During training, it’s important to log metrics like loss values so you can monitor progress.  

This helper function prints metrics from **every worker**:  
- Collects the current **loss** and **epoch** into a dictionary.  
- Uses `ray.train.get_context().get_world_rank()` to identify which worker is reporting.  
- Prints the metrics along with the worker’s rank for debugging and visibility.   

In [ ]:
# 10. Report training metrics from each worker

def print_metrics_ray_train(loss: torch.Tensor, epoch: int) -> None:
    metrics = {"loss": loss.item(), "epoch": epoch}  
    world_rank = ray.train.get_context().get_world_rank() # report from all workers
    print(f"{metrics=} {world_rank=}")
    return metrics

<div class="alert alert-block alert-info">

If you want to log only from the rank 0 worker, use this code:

```python
def print_metrics_ray_train(loss: torch.Tensor, epoch: int) -> None:
    metrics = {"loss": loss.item(), "epoch": epoch}
    if ray.train.get_context().get_world_rank() == 0:  # report only from the rank 0 worker
        print(f"{metrics=} {world_rank=}")
    return metrics
```

</div>

## 11 · Save Checkpoints and Report Metrics  

We will report intermediate metrics and checkpoints using the [`ray.train.report`](https://docs.ray.io/en/latest/train/api/doc/ray.train.report.html#ray-train-report) utility function.  

This helper function:  
- Creates a temporary directory to stage the checkpoint.  
- Saves the model weights with `torch.save()`.  
  * Since the model is wrapped in **DistributedDataParallel (DDP)**, we call `model.module.state_dict()` to unwrap it.  
- Calls `ray.train.report()` to:  
  * Log the current metrics (e.g., loss, epoch).  
  * Attach a `Checkpoint` object created from the staged directory.  

This way, each epoch produces both **metrics for monitoring** and a **checkpoint for recovery or inference**.  

In [ ]:
# 11. Save checkpoint and report metrics with Ray Train

def save_checkpoint_and_metrics_ray_train(model: torch.nn.Module, metrics: dict[str, float]) -> None:
    # Create a temporary directory to stage checkpoint files
    with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
        # Save the model weights.
        # Note: under DDP the model is wrapped in DistributedDataParallel,
        # so we unwrap it with `.module` before calling state_dict().        
        torch.save(
            model.module.state_dict(),  # note the `.module` to unwrap the DistributedDataParallel
            os.path.join(temp_checkpoint_dir, "model.pt"),
        )
        
        # Report metrics and attach a checkpoint to Ray Train.
        # → metrics are logged centrally
        # → checkpoint allows resuming training or running inference later
        ray.train.report(
            metrics,
            checkpoint=ray.train.Checkpoint.from_directory(temp_checkpoint_dir),
        )

<div class="alert alert-block alert-info">
  <p><strong>Quick notes:</strong></p>
  <ul>
    <li>
      Use 
      <a href="https://docs.ray.io/en/latest/train/api/doc/ray.train.report.html#ray.train.report" target="_blank">
        ray.train.report
      </a> to save the metrics and checkpoint.
    </li>
    <li>Only metrics from the rank 0 worker are reported.</li>
  </ul>
</div>

### Note on the Checkpoint Lifecycle  

The diagram above shows how a checkpoint moves from **local storage** (temporary directory on a worker) to **persistent cluster or cloud storage**.  

Key points to remember:  
- Since the model is identical across all workers, it’s enough to **write the checkpoint only on the rank-0 worker**.  
  * However, you still need to call [`ray.train.report`](https://docs.ray.io/en/latest/train/api/doc/ray.train.report.html#ray-train-report) on **all workers** to keep the training loop synchronized.  
- Ray Train expects every worker to have access to the **same persistent storage location** for writing files.  
- For production jobs, **cloud storage** (e.g., S3, GCS, Azure Blob) is the recommended target for checkpoints.  


<img src="https://docs.ray.io/en/latest/_images/checkpoint_lifecycle.png" width=800>


### 12 · Save Checkpoints on Rank-0 Only  

To avoid redundant writes, we update the checkpointing function so that **only the rank-0 worker** saves the model weights.  

- **Temporary directory** → Each worker still creates a temp directory, but only rank-0 writes the model file.  
- **Rank check** → `ray.train.get_context().get_world_rank()` ensures that only worker 0 performs the checkpointing.  
- **All workers report** → Every worker still calls [`ray.train.report`](https://docs.ray.io/en/latest/train/api/doc/ray.train.report.html#ray-train-report), but only rank-0 attaches the actual checkpoint. This keeps the training loop synchronized.  

This pattern is the recommended best practice:  
- Avoids unnecessary duplicate checkpoints from multiple workers.  
- Still guarantees that metrics are reported from every worker.  
- Ensures checkpoints are cleanly written once per epoch to persistent storage.  

In [ ]:
# 12. Save checkpoint only from the rank-0 worker

def save_checkpoint_and_metrics_ray_train(model: torch.nn.Module, metrics: dict[str, float]) -> None:
    with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
        checkpoint = None

        # Only the rank-0 worker writes the checkpoint file
        if ray.train.get_context().get_world_rank() == 0:
            torch.save(
                model.module.state_dict(),  # unwrap DDP before saving
                os.path.join(temp_checkpoint_dir, "model.pt"),
            )
            checkpoint = ray.train.Checkpoint.from_directory(temp_checkpoint_dir)

        # All workers still call ray.train.report()
        # → keeps training loop synchronized
        # → metrics are logged from each worker
        # → only rank-0 attaches a checkpoint
        ray.train.report(
            metrics,
            checkpoint=checkpoint,
        )


Check our guide on [saving and loading checkpoints](https://docs.ray.io/en/latest/train/user-guides/checkpoints.html) for more details and best practices.

### 13 · Configure Persistent Storage with `RunConfig`  

To tell Ray Train **where to store results, checkpoints, and logs**, we use a [`RunConfig`](https://docs.ray.io/en/latest/train/api/doc/ray.train.RunConfig.html).  

- **`storage_path`** → Base directory for all outputs of this training run.  
  * In this example we use `/mnt/cluster_storage/training/`, which is **persistent shared storage** across all nodes.  
  * This ensures checkpoints and metrics remain available even after the cluster shuts down.  
- **`name`** → A human-readable name for the run (e.g., `"distributed-mnist-resnet18"`). This is used to namespace output files.  

Together, the `RunConfig` defines how Ray organizes and persists all artifacts from your training job.  

In [ ]:
# 13. Configure persistent storage and run name

storage_path = "/mnt/cluster_storage/training/"
run_config = RunConfig(
    storage_path=storage_path,         # where to store checkpoints/logs
    name="distributed-mnist-resnet18"  # identifier for this run
)

### Distributed Data-Parallel Training with Ray Train  

This diagram shows the same DDP workflow as before, but now with **Ray Train utilities** highlighted:  

1. **`ray.train.torch.prepare_data_loader()`**  
   - Automatically wraps your PyTorch DataLoader with a `DistributedSampler`.  
   - Ensures each worker processes a unique shard of the dataset.  
   - Moves batches to the correct device (GPU or CPU).  

2. **`ray.train.torch.prepare_model()`**  
   - Moves your model to the right device.  
   - Wraps it in `DistributedDataParallel (DDP)` so gradients are synchronized across workers.  
   - Removes the need for manual `.to("cuda")` calls or DDP boilerplate.  

3. **`ray.train.report()`**  
   - Centralized way to report metrics and attach checkpoints.  
   - Keeps the training loop synchronized across all workers, even if only rank-0 saves the actual checkpoint.  

By combining these helpers, Ray Train takes care of the **data sharding, model replication, gradient synchronization, and checkpoint lifecycle** — letting you keep your training loop clean and close to standard PyTorch.  


|<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-ai-libraries/diagrams/multi_gpu_pytorch_annotated_v5.png" width="70%" loading="lazy">|
|:--|
||

## 14 · Create the `TorchTrainer`  

Now we bring everything together with a [`TorchTrainer`](https://docs.ray.io/en/latest/train/api/doc/ray.train.torch.TorchTrainer.html).  

The `TorchTrainer` is the high-level Ray Train class that:  
- Launches the per-worker training loop (`train_loop_ray_train`) across the cluster.  
- Applies the scaling setup from `scaling_config` (number of workers, GPUs/CPUs).  
- Uses `run_config` to decide where results and checkpoints are stored.  
- Passes `train_loop_config` (hyperparameters like `num_epochs` and `global_batch_size`) into the training loop.  

This object encapsulates the **distributed orchestration**, so you can start training with a simple call to `trainer.fit()`.  

In [ ]:
# 14. Set up the TorchTrainer

trainer = TorchTrainer(
    train_loop_ray_train,          # training loop to run on each worker
    scaling_config=scaling_config, # number of workers and resource config
    run_config=run_config,         # storage path + run name for artifacts
    train_loop_config=train_loop_config,  # hyperparameters passed to the loop
)

### 15 · Launch Training with `trainer.fit()`  

Calling `trainer.fit()` starts the distributed training job and blocks until it completes.  

When the job launches, you’ll see logs that confirm:  
- **Process group setup** → Ray initializes a distributed worker group and assigns ranks (e.g., `world_rank=0` and `world_rank=1`).  
- **Worker placement** → Each worker is launched on a specific node and device. The logs show IP addresses, process IDs, and rank assignments.  
- **Model preparation** → Each worker moves the model to its GPU (`cuda:0`) and wraps it in **DistributedDataParallel (DDP)**.  

These logs are a quick sanity check that Ray Train is correctly orchestrating multi-GPU training across your cluster.  

|<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-intro/ray-train-intro-logs.png" width="80%" loading="lazy">|
|:--|
||

In [ ]:
# 15. Launch distributed training

# trainer.fit() starts the training job:
# - Spawns workers according to scaling_config
# - Runs train_loop_ray_train() on each worker
# - Collects metrics and checkpoints into result
result = trainer.fit()

## 16 · Inspect the Training Results  

When `trainer.fit()` finishes, it returns a [`Result`](https://docs.ray.io/en/latest/train/api/doc/ray.train.Result.html) object.  

This object contains:  
- **Final metrics** → the most recent values reported from the training loop (e.g., loss at the last epoch).  
- **Checkpoint** → a reference to the latest saved checkpoint, including its path in cluster storage.  
- **Metrics dataframe** → a history of all reported metrics across epochs (accessible with `result.metrics_dataframe`).  
- **Best checkpoints** → Ray automatically tracks checkpoints associated with their reported metrics.  

In the output above, you can see:  
- The final reported loss at epoch 1.  
- The location where checkpoints are stored (`/mnt/cluster_storage/training/distributed-mnist-resnet18/...`).  
- A list of best checkpoints with their corresponding metrics.  

This makes it easy to both **analyze training performance** and **restore the trained model** later for inference.  

In [ ]:
# 16. Show the training results  

result  # contains metrics, checkpoints, and run history

### 17 · View Metrics as a DataFrame  

The `Result` object also includes a `metrics_dataframe`, which stores the full history of metrics reported during training.  

- Each row corresponds to one reporting step (here, each epoch).  
- The columns show the metrics you logged in the training loop (e.g., `loss`, `epoch`).  
- This makes it easy to plot learning curves or further analyze training progress.  

In the example below, you can see the training loss steadily decreasing across two epochs.  

In [ ]:
# 17. Display the full metrics history as a pandas DataFrame

result.metrics_dataframe

<div class="alert alert-block alert-info">
  <p>
    To learn more about the training results, see this 
    <a href="https://docs.ray.io/en/latest/train/user-guides/results.html" target="_blank">
      docs
    </a> on inspecting the training results.
  </p>
</div>

## 18 · Load a Checkpoint for Inference  

After training, we often want to reload the model and use it for predictions.  
Here we define a **Ray actor** (`ModelWorker`) that loads the checkpointed ResNet-18 onto a GPU and serves inference requests.  

- **Initialization (`__init__`)**:  
  * Reads the checkpoint directory using `checkpoint.as_directory()`.  
  * Loads the model weights into a fresh ResNet-18.  
  * Moves the model to GPU and sets it to evaluation mode.  

- **Prediction (`predict`)**:  
  * Accepts either a single image (`[C,H,W]`) or a batch (`[B,C,H,W]`).  
  * Ensures the tensor is correctly shaped and moved to GPU.  
  * Runs inference in `torch.inference_mode()` for efficiency.  
  * Returns the predicted class indices as a Python list.  

Finally, we launch the actor with `ModelWorker.remote(result.checkpoint)`.  
This spawns a dedicated process with **1 GPU** attached that can serve predictions using the trained model.  

In [ ]:
# 18. Define a Ray actor to load the trained model and run inference

@ray.remote(num_gpus=1)  # allocate 1 GPU to this actor
class ModelWorker:
    def __init__(self, checkpoint):
        # Load model weights from the Ray checkpoint (on CPU first)
        with checkpoint.as_directory() as ckpt_dir:
            model_path = os.path.join(ckpt_dir, "model.pt")
            state_dict = torch.load(
                model_path,
                map_location=torch.device("cpu"),
                weights_only=True,
            )
        # Rebuild the model, load weights, move to GPU, and set to eval mode
        self.model = build_resnet18()
        self.model.load_state_dict(state_dict)
        self.model.to("cuda")
        self.model.eval()

    @torch.inference_mode()  # disable autograd for faster inference
    def predict(self, batch):
        """
        batch: torch.Tensor or numpy array with shape [B,C,H,W] or [C,H,W]
        returns: list[int] predicted class indices
        """
        x = torch.as_tensor(batch)
        if x.ndim == 3:          # single image → add batch dimension
            x = x.unsqueeze(0)   # shape becomes [1,C,H,W]
        x = x.to("cuda", non_blocking=True)

        logits = self.model(x)
        preds = torch.argmax(logits, dim=1)
        return preds.detach().cpu().tolist()

# Create a fresh actor instance (avoid naming conflicts)
worker = ModelWorker.remote(result.checkpoint)

### 19 · Run Inference and Visualize Predictions  

With the `ModelWorker` actor running on GPU, we can now generate predictions on random samples from the MNIST dataset and plot them.  

Steps in this cell:  
1. **Normalization on CPU**  
   - Convert each image to a tensor with `ToTensor()`.  
   - Apply channel-specific normalization (`0.5` mean / std).  
   - Keep this preprocessing on CPU for efficiency.  

2. **Prediction on GPU via Actor**  
   - Each normalized image is expanded to shape `[1, C, H, W]`.  
   - The tensor is sent to the remote `ModelWorker` for inference.  
   - `ray.get(worker.predict.remote(x))` retrieves the predicted class index.  

3. **Plot Results**  
   - Display a 3×3 grid of random MNIST samples.  
   - Each subplot shows the **true label** and the **predicted label** from the trained ResNet-18.  

This demonstrates a simple but practical workflow: **CPU-based preprocessing + GPU-based inference in a Ray actor**.  

In [ ]:
# 19. CPU preprocessing + GPU inference via Ray actor

to_tensor = ToTensor()

def normalize_cpu(img):
    # Convert image (PIL) to tensor on CPU → shape [C,H,W]
    t = to_tensor(img)                # [C,H,W] on CPU
    C = t.shape[0]
    # Apply channel-wise normalization (grayscale vs RGB)
    if C == 3:
        norm = Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    else:
        norm = Normalize((0.5,), (0.5,))
    return norm(t)

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3

# Plot a 3x3 grid of random MNIST samples with predictions
for i in range(1, cols * rows + 1):
    idx = np.random.randint(0, len(dataset))
    img, label = dataset[idx]

    # Preprocess on CPU, add batch dim → [1,C,H,W]
    x = normalize_cpu(img).unsqueeze(0)    

    # Run inference on GPU via Ray actor, fetch result   
    pred = ray.get(worker.predict.remote(x))[0]  # int
    
    # Plot image with true label and predicted label
    figure.add_subplot(rows, cols, i)
    plt.title(f"label: {label}; pred: {int(pred)}")
    plt.axis("off")
    arr = np.array(img)
    plt.imshow(arr, cmap="gray" if arr.ndim == 2 else None)

plt.tight_layout()
plt.show()

### 20 · Clean Up the Ray Actor  

Once you’re done running inference, it’s a good practice to free up resources:  

- **`ray.kill(worker, no_restart=True)`** → stops the `ModelWorker` actor and releases its GPU.  
- **`del worker` + `gc.collect()`** → drop local references so Python’s garbage collector can clean up.  

This ensures the GPU is no longer pinned by the actor and can be reused for other jobs.  

In [ ]:
# 20.

# stop the actor process and free its GPU
ray.kill(worker, no_restart=True)     

# drop local references so nothing pins it
del worker

# Forcing garbage collection is optional:
# - Cluster resources are already freed by ray.kill()
# - Python will clean up the local handle eventually
# - gc.collect() is usually unnecessary unless debugging memory issues
gc.collect()